In [ ]:
#default_exp training

In [ ]:
#export
from speechsep.core import *
from speechsep.masks import *
from speechsep.pipe import *
from speechsep.utils import *
from speechsep.data import *
from speechsep.imports import *
from speechsep.models import *

In [ ]:
#export
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision

In [ ]:
#export
fn = Path("../data/esc50_sample/")
pipe = AudioPipe(fn)

# Dataset/DataLoader

In [ ]:
#export
class Tensorify(Transform):
    def encodes(self, x):
        tnsr = complex2real(x.data) if hasattr(x, "data") else complex2real(x)
        return torch.FloatTensor(tnsr)

class AudioDataset(Dataset):
    @delegates(AudioPipe)
    def __init__(self, fn, **kwargs):
        self.fn = fn
        self.pipe = AudioPipe(fn, **kwargs)
        self.n_samples = len(get_audio_files(fn))
    
    def __getitem__(self, index):
        x,y = self.pipe(index)
        x,y = Tensorify()(x),Tensorify()(y)
        return x,y
    
    def __len__(self):
        return self.n_samples

# Loss Function

In [ ]:
#export
def ext_MSELoss():
    return sumnn.MSELoss(sep, yb)

def loss_func(sep, yb):
    return min(,nn.MSELoss(sep[:,:,-1], yb))

# Training Loop

In [ ]:
#export
bs = 1
shuffle=True
workers=2

In [ ]:
#export
dataset = AudioDataset(fn)
dataloader = DataLoader(dataset=dataset, batch_size=bs, shuffle=shuffle, num_workers=workers)

dataiter = iter(dataloader)

In [ ]:
#export
data = dataiter.next()
data;

In [ ]:
#export
n_epochs = 1
n_samples = len(dataset)
n_iter = math.ceil(n_samples/bs)

In [ ]:
#export
model = U_Net(img_ch=2, output_ch=4)
model.train();

In [ ]:
#export
for epoch in range(n_epochs):
    for i, (xb, yb) in enumerate(dataloader):
        out = model(xb)
        mask1 = MaskcIRM(out[:,:2,:,:])
        mask2 = MaskcIRM(out[:,2:,:,:])
        sep = mask1*xb, mask2*xb
        loss = loss_func(sep, yb)
        if (i+1)%5==0:
            print(f'epoch {epoch}: step {(i+1)/n_iter}')
        #loss.backward()
        #optimizer.step()
        #optimizer.zero_grad()

True


/home/holydemon/.local/anaconda3/envs/fai2/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


TypeError: tuple indices must be integers or slices, not tuple